# Joy Recognition Model

In [1]:
import math
import numpy as np
import pandas as pd

from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

#https://scikit-learn.org/stable/modules/neural_networks_supervised.html
from sklearn.neural_network import MLPClassifier

In [2]:
#Want to get confidence level greater or equal to 0.85
def filterConfidenceLevel(df):
    return df[df[' confidence'] >= 0.85]

#Change label 'Happy' to 1, return numpy array, rest as 0 as in "Not happy"
def labelHappiness(df):
    df.loc[df.label == "angry", "cluster"] = 0
    df.loc[df.label == "disgust", "cluster"] = 0
    df.loc[df.label == "fear", "cluster"] = 0
    df.loc[df.label == "happy", "cluster"] = 1
    df.loc[df.label == "neutral", "cluster"] = 0
    df.loc[df.label == "sad", "cluster"] = 0
    df.loc[df.label == "surprise", "cluster"] = 0
    return df['cluster']

def addLabel(df, label):
    df = df['label'] = label
    return df

In [104]:
#Data
fer2013 = pd.read_csv('./dataset/training/fer2013.csv')
phoebe = pd.read_csv('./dataset/testing/phoebe.csv')
angryTrainingSet = pd.read_csv('./dataset/training/emotioncsv/angrycsv/angry.csv')
disgustTrainingSet = pd.read_csv('./dataset/training/emotioncsv/disgustcsv/disgust.csv')
fearTrainingSet = pd.read_csv('./dataset/training/emotioncsv/fearcsv/fear.csv')
happyTrainingSet = pd.read_csv('./dataset/training/emotioncsv/happycsv/happy.csv')
neutralTrainingSet = pd.read_csv('./dataset/training/emotioncsv/neutralcsv/neutral.csv')
sadTrainingSet = pd.read_csv('./dataset/training/emotioncsv/sadcsv/sad.csv')
surpriseTrainingSet = pd.read_csv('./dataset/training/emotioncsv/surprisecsv/surprise.csv')
testMaskedData = pd.read_csv('./dataset/testing/TestMaskedPhotosV1.csv')
testMaskedDataV2 = pd.read_csv('./dataset/testing/TestMaskedPhotosV2.csv')

# Preprocessing

In [105]:
# requiredAUs = [' AU06_r', ' AU06_c', ' AU10_c', ' AU10_r', ' AU12_c', ' AU12_r', ' AU14_c', ' AU14_r']

#Upper Of the face
requiredAUs = [' AU01_r', ' AU01_c', ' AU02_c', ' AU02_r', ' AU04_r', ' AU04_c', ' AU05_c', ' AU05_r', ' AU06_r', ' AU06_c', ' AU07_r', ' AU07_c']
forFilteringAndLabeling = [' confidence', 'label'] 

In [106]:
#Add Labels
angryTrainingSet['label'] = 'angry'
disgustTrainingSet['label'] = 'disgust'
fearTrainingSet['label'] = 'fear'
happyTrainingSet['label'] = 'happy'
neutralTrainingSet['label'] = 'neutral'
sadTrainingSet['label'] = 'sad'
surpriseTrainingSet['label'] = 'surprise'

combineEmotions = pd.concat([angryTrainingSet, disgustTrainingSet, 
                             fearTrainingSet, happyTrainingSet, neutralTrainingSet,
                            sadTrainingSet, surpriseTrainingSet], axis = 0)

#Get all initial required columns
combineEmotions = combineEmotions[requiredAUs + forFilteringAndLabeling]

#Get Confidence level >= 0.85
combineEmotions = filterConfidenceLevel(combineEmotions)

#Response Value -> Numpy array
y = labelHappiness(combineEmotions)

#Dropping unnecessary columns, only AUs for 6_c, 6_r, 12_c, and 12_r needed
combineEmotions = combineEmotions.drop(columns = forFilteringAndLabeling + ['cluster'])

#Features -> Numpy array
X = combineEmotions.to_numpy()

# Training Model

In [107]:
#Training the model 
#using 'adam' because its good for large training sets
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, max_iter = 200)
clf.fit(X,y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1)

In [110]:
testMaskedDataV2 = testMaskedDataV2[testMaskedDataV2['label'] == 'neutral']
testMaskedDataV2 = testMaskedDataV2[testMaskedDataV2[' confidence'] >= 0.95]
X = testMaskedDataV2[requiredAUs].to_numpy()

In [111]:
clf.predict(X)

ValueError: Found array with 0 sample(s) (shape=(0, 12)) while a minimum of 1 is required.